# Simulated annealing
Take sequence and swap to random elements in it. Repeat this from temperature start to end with cooling rate. On each temperature do some steps with random swaping.\
(+) can improve previous scores\
(-) still can be in loval optmum, instead of global\
(-) dependent on temperature, time consuming

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
import random
import time
import math

In [8]:
# we will be improving beam search desicion
df = pd.read_csv('/kaggle/input/beam-search/submission_beam.csv')

In [9]:
df

,id,text
0,0,reindeer mistletoe scrooge elf gingerbread chi...
1,1,reindeer mistletoe elf scrooge gingerbread chi...
2,2,nutcracker yuletide grinch gifts holiday decor...
3,3,ornament yuletide of the holiday decorations a...
4,4,eggnog fruitcake chocolate poinsettia peppermi...
5,5,poinsettia yuletide eggnog fruitcake mistletoe...


In [4]:
from metric import PerplexityCalculator
scorer = PerplexityCalculator('/kaggle/input/gemma-2/transformers/gemma-2-9b/2')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

* ">" new version with score improved
* "<" new version with poorer score
* "-" maintaining existing version

In [10]:
temp_start = 10.0    #how high a temperature we start with (prior 10)
temp_end = 0.5       #final temperature (prior 0.2)
cooling_rate = 0.95  #how quick we cool each time we drop temp (prior 0.95)
steps_per_temp = 20 #5  #steps at each temperature (prior 20)    <---- Increase this for a longer run (20 steps is about 3 hours)

def simulated_annealing_optimize(text: str, temp_start=temp_start, temp_end=temp_end, cooling_rate=cooling_rate, steps_per_temp=steps_per_temp, verbose=False):
    """Optimize word sequence using simulated annealing, handling NaN scores by randomizing.

    Args:
       text: Input string of space-separated words to optimize
       temp_start: Starting temperature - higher means more random exploration
       temp_end: Ending temperature - lower means more selective at end
       cooling_rate: How fast temperature decreases each step
       steps_per_temp: How many swaps to try at each temperature
       verbose: Whether to print detailed progress
    """
    
    words = text.split()

    current = words.copy()
    current_score = scorer.get_perplexity(' '.join(current))

    # Handling any NaNs...
    if math.isnan(current_score):
        # Keep shuffling until we find a valid sequence
        while True:
            current = words.copy()
            random.shuffle(current)
            current_score = scorer.get_perplexity(' '.join(current))
            if not math.isnan(current_score):
                break
            
    best = current.copy()
    best_score = current_score
    temp = temp_start
    print(f"Start Temperature: {temp:.2f}, Initial score: {current_score:.2f}")
    
    # Main annealing loop - keep trying until we've cooled down enough
    while temp > temp_end:
        for _ in range(steps_per_temp):  # Do multiple attempts at each temperature
            # Try improving sequence by swapping random pairs of words
            i, j = random.sample(range(len(words)), 2)
            neighbor = current.copy()
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            
            # Get score for this arrangement, skip if invalid
            neighbor_score = scorer.get_perplexity(' '.join(neighbor))
            if math.isnan(neighbor_score):
                continue
            
            # Accept better scores, sometimes accept worse ones based on temperature
            delta = neighbor_score - current_score
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current = neighbor
                current_score = neighbor_score
                
                if current_score < best_score:
                    best = current.copy()
                    best_score = current_score
                    print(">", end="")
                else: print("<", end="")
            else:print("-", end="")

        
        # Reduce temperature according to cooling schedule (AFTER all steps at this temperature)
        temp *= cooling_rate
        if verbose: print(f"\nTemperature: {temp:.2f}, Current score: {current_score:.2f}")
    
    print(f"\nFinal score: {best_score:.2f}, {best}")
    
    return ' '.join(best), best_score

In [11]:
submission = pd.DataFrame(columns=['id', 'text'])
scores = []

# Process each sample
for idx, row in df.iterrows():
   print(f"\nProcessing sample {idx}...")
   optimized, score = simulated_annealing_optimize(row.text)
   scores.append(score)
   
   # Add to submission dataframe
   submission.loc[idx] = {
       'id': row.id,
       'text': optimized
   }
   print("-" * 50)

# Print summary statistics
print("\nScore Summary:")
print(f"Submission mean score: {np.mean(scores):.2f}")

# Save to CSV
submission.to_csv("submission_simulated_annealing.csv", index=False)
print("\nSubmission file created!")

submission


Processing sample 0...
Start Temperature: 10.00, Initial score: 515.99
------------------------------------------<-----------<-------------<<--->---------------------------------------------------------------------------------------------<-----------------------------------------------------------------------------------------------------------------------------------------------<----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,id,text
0,0,reindeer mistletoe elf scrooge gingerbread chi...
1,1,reindeer mistletoe elf scrooge gingerbread chi...
2,2,magi yuletide cheer holiday gifts decorations ...
3,3,ornament yuletide holiday the of decorations a...
4,4,eggnog fruitcake chocolate poinsettia peppermi...
5,5,poinsettia yuletide eggnog fruitcake mistletoe...
